# VGG16 fine-tuning for defect detection

This section builds a VGG16 model with ImageNet weights, lets you choose any valid input shape (HxWx3), and fine-tune only the last layers you specify.

In [ ]:
from keras import Model
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.layers import BatchNormalization, Dense, Dropout, GlobalAveragePooling2D, Input

In [ ]:
def build_vgg16_finetune(
    input_shape=(128, 128, 3),
    num_classes=2,
    train_last_n_layers=4,
    base_trainable=False,
    dropout_rate=0.2,
    l2_reg=0.0,
):
    """
    Build a VGG16-based model with ImageNet weights and a custom classification head.

    Parameters
    ----------
    input_shape : (H, W, 3)
        Model input shape. Must have 3 channels.
    num_classes : int
        Number of output classes.
    train_last_n_layers : int
        Number of layers (from the end of the base model) to unfreeze for fine-tuning.
    base_trainable : bool
        If True, allow training on selected last N layers of VGG16.
    dropout_rate : float
        Dropout after the pooled features (0 disables).
    l2_reg : float
        L2 weight decay for the dense head (0 disables).

    Returns
    -------
    keras.Model
        Compiled model ready to train.
    """
    
    assert input_shape[-1] == 3, "Input must have 3 channels (RGB)."

    # Load VGG16 base with ImageNet weights and no top
    base = VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape,
    )

    # Freeze all layers by default
    base.trainable = False

    # Optionally unfreeze last N layers
    if base_trainable and train_last_n_layers > 0:
        # Unfreeze only the last N layers of the base
        for layer in base.layers[-train_last_n_layers:]:
            if not isinstance(layer, BatchNormalization):
                layer.trainable = True

    # Build head
    inputs = Input(shape=input_shape)
    x = base(inputs, training=False)  # important for BN layers in eval mode
    x = GlobalAveragePooling2D(name="gap")(x)
    if dropout_rate > 0:
        x = Dropout(dropout_rate)(x)
    kernel_reg = l2(l2_reg) if l2_reg > 0 else None
    x = Dense(256, activation="relu", kernel_regularizer=kernel_reg)(x)
    x = Dropout(dropout_rate)(x) if dropout_rate > 0 else x

    outputs = Dense(num_classes, activation="softmax", name="predictions")(x)
    model = Model(inputs, outputs, name="vgg16_finetune")

    optimizer = Adam(learning_rate=1e-3)
    
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [ ]:
model = build_vgg16_finetune(
    input_shape=(128, 128, 3),
    num_classes=2,
    train_last_n_layers=6,
    base_trainable=True,
    dropout_rate=0.3,
    l2_reg=1e-4,
)

In [ ]:
model.summary()